In [3]:
## Notebook file for testing and playing around with methods, have fun! ##

In [1]:
# Import relevant packages.
from bs4 import BeautifulSoup 
import requests
import urllib3
from datetime import datetime
import pandas as pd
import json
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# Using Requests & BeautifulSoup, pass a URL to this method and it will return the URL's HTML code as a list of lines.
def get_html_rows(url):
    r = requests.get(url, verify=False) 
    s = BeautifulSoup(r.content, 'html5lib') 
    html = s.prettify()
    return html.splitlines()

In [3]:
# Taking 'https://www.wta.org/go-hiking/hikes' as a parameter,
# this method will find all links to pages consisting of a list of hikes and, return all links as a list using recursion.
# Note, this method does not include 'https://www.wta.org/go-outside/hikes' in the returned list.
def get_hikes_list_page_urls(url, last_page=None):
    
    html_rows = get_html_rows(url)
    
    check = False
    exit = False
    active_page = 0
    index_start = 0
    index_end = 0

    itr = -1
    for row in html_rows:
        itr += 1

        if '"active"' in row:
            index_start = html_rows.index(row)
            active_page = int(html_rows[itr + 2].lstrip())

        if '"last"' in row:
            index_end = html_rows.index(row)
            last_page = int(html_rows[itr + 2].lstrip())
            check = True
            break           
        elif check == False and '"next"' in row:
            index_end = html_rows.index(row)
            last_page = int(html_rows[itr - 3].lstrip())
            check = True
            exit = True
            break        
        elif check == False and active_page == last_page:
            return
            
    rows_range = html_rows[index_start : index_end]
    pages_found = [item[item.find('https') : item.find('">')] for item in rows_range if 'www.wta.org' in item]
    next_page = pages_found[0]
    
    if exit == True:
        return pages_found
    else:
        return list(set().union(pages_found, get_hikes_list_page_urls(next_page, last_page)))

In [4]:
# Taking a list of links consisting of WTA's pages, that houses their lists of hikes, as a parameter,
# this method will return a list of links to all hikes found on wta.org.    
def get_individual_hike_urls(hikes_list_page_urls):
    
    hike_urls_list = []
    
    for url in hikes_list_page_urls:
        html_rows = get_html_rows(url)
        
        for row in html_rows:
            
            if "listitem-title" in row:
                hike_url = row[row.find('https') : row.find('" title=')]                
                hike_urls_list.append(hike_url)
                
    return hike_urls_list

Currently grabbing: TITLE,REGION,DISTANCE,DIST_TYPE,GAIN,HIGHEST,RATING,RATING_COUNT,LATITUDE,LONGITUDE,REPORT_DATE,REPORT_COUNT,URL

Want to grab:
TITLE, GENERAL AREA, SPECIFIC AREA, LONGITUDE, LATITUDE, DISTANCE, DISTANCE TYPE, ELEVATION GAIN, HIGHEST POINT, CURRENT-RATING, ALL FEATURES, PARKING PASS/ENTRY FEE

In [5]:
# Taking a list of individual hike links as a parameter,
# this method will reutrn specific information for each hike in the format of a DataFrame.
def get_hike_info(hike_urls):
    
    # Create the dataframe
    df = pd.DataFrame()

    titles = []
    regions = []
    specific_regions = [] # NEW
    distances = []
    dist_types = []
    gains = []
    highests = []
    ratings = []
    rating_counts = []
    latitudes = []
    longitudes = []
    report_counts = []
    report_dates = []
    hike_links = []
    coast = []
    rivers = []
    lakes = []
    waterfalls = []
    old_growth = []
    fall_foliage = []
    wildflowers = []
    mountain_views = []
    summits = []
    wildlife = []
    ridges = []
    campsites = []
    dogs_on_leash = []
    no_dogs = []
    kid_friendly = []
    parking_entry_fee = []

    rownum = 1
    PARKING_PASS_FOUND = False
    for url in hike_urls:
        print(url)
        hike_html_rows = get_html_rows(url)

        itr1 = -1
        for row in hike_html_rows:
            itr1 += 1

            if '"documentFirstHeading"' in row:
                hike_title = hike_html_rows[itr1 + 1].lstrip()
                titles.append(hike_title)

            if 'Location' in row:
                hike_region_full = hike_html_rows[itr1 + 3].lstrip()
                if hike_region_full is not None:
                    hike_region = re.findall('([\s\S]*) --', hike_region_full)
                    if len(hike_region) == 0:
                        regions.append(hike_region_full)
                    else:
                        regions.append(hike_region[0])

                    specific_region = re.findall("-- ([\s\S]*)", hike_region_full)
                    if len(specific_region) == 0:
                        specific_regions.append(None)
                    else:
                        specific_regions.append(specific_region[0])
                
            # Looking for the other misc features
            if '"Coast"' in row:
                coast.append(1)
                    
            if '"Rivers"' in row:
                rivers.append(1)
                
            if '"Lakes"' in row:
                lakes.append(1)
                
            if '"Waterfalls"' in row:
                waterfalls.append(1)
                
            if '"Old growth"' in row:
                old_growth.append(1)
            
            if '"Fall foliage"' in row:
                fall_foliage.append(1)
                
            if '"Wildflowers/Meadows"' in row:
                wildflowers.append(1)

            if '"Mountain views"' in row:
                mountain_views.append(1)
                
            if '"Summits"' in row:
                summits.append(1)
                
            if '"Wildlife"' in row:
                wildlife.append(1)
                
            if '"Ridges/passes"' in row:
                ridges.append(1)
                
            if '"Established campsites"' in row:
                campsites.append(1)
                
            if '"Dogs allowed on leash"' in row:
                dogs_on_leash.append(1)
                
            if '"Dogs not allowed"' in row:
                no_dogs.append(1)
                
            if '"Good for kids"' in row:
                kid_friendly.append(1)

            if '"distance"' in row:
                hike_distance_string = hike_html_rows[itr1 + 2].lstrip()
                hike_distance = float(hike_distance_string[ : hike_distance_string.find(' mile')])
                if ',' in hike_distance_string:
                    hike_distance_type = hike_distance_string[hike_distance_string.find(', ') + 2 : ]
                elif 'of trails' in hike_distance_string:
                    hike_distance_type = hike_distance_string[hike_distance_string.find('of trails') + 3 : ]
                else:
                    hike_distance = 'ERROR'
                distances.append(hike_distance)
                dist_types.append(hike_distance_type)

            if 'Gain:' in row:
                hike_gain = float(hike_html_rows[itr1 + 2].lstrip())
                gains.append(hike_gain)

            if 'Highest Point:' in row:
                hike_highest = float(hike_html_rows[itr1 + 2].lstrip())
                highests.append(hike_highest)

            if '"current-rating"' in row:
                rating_string = hike_html_rows[itr1 + 1].lstrip()
                hike_rating = float(rating_string[ : rating_string.find(' out of')])
                ratings.append(hike_rating)

            if '"rating-count"' in row:
                rating_count_string = hike_html_rows[itr1 + 1].lstrip()
                rating_count = int(rating_count_string[rating_count_string.find('(') + 1 : rating_count_string.find(' vote')])
                rating_counts.append(rating_count)
                
            if '<script type="application/ld+json">' in row:
                json_string = hike_html_rows[itr1 + 1].lstrip()
                hike_json = json.loads(json_string)
                try:
                    latitude = hike_json['geo']['latitude']
                    longitude = hike_json['geo']['longitude']
                    latitudes.append(latitude)
                    longitudes.append(longitude)
                except:
                    pass
                
            if 'Parking Pass/Entry Fee' in row:
                if PARKING_PASS_FOUND == False:
                    PARKING_PASS_FOUND = True
                    required_pass = hike_html_rows[itr1 + 3].lstrip()
                    if required_pass == "None":
                        required_pass = "No pass"
                    parking_entry_fee.append(required_pass)

                
        if len(titles) != rownum:
            titles.append(None)

        if len(regions) != rownum:
            regions.append(None)
            
        if len(specific_regions) != rownum:
            specific_regions.append(None)

        if len(distances) != rownum:
            distances.append(None)

        if len(dist_types) != rownum:
            dist_types.append(None)

        if len(gains) != rownum:
            gains.append(None)

        if len(highests) != rownum:
            highests.append(None)

        if len(ratings) != rownum:
            ratings.append(None)

        if len(rating_counts) != rownum:
            rating_counts.append(None)
            
        if len(latitudes) != rownum:
            latitudes.append(None)

        if len(longitudes) != rownum:
            longitudes.append(None)
        
        if len(coast) == 0:
            coast.append(0)
            
        if len(rivers) == 0:
            rivers.append(0)
        
        if len(lakes) == 0:
            lakes.append(0)
        
        if len(waterfalls) == 0:
            waterfalls.append(0)
            
        if len(old_growth) == 0:
            old_growth.append(0)
            
        if len(fall_foliage) == 0:
            fall_foliage.append(0)
            
        if len(wildflowers) == 0:
            wildflowers.append(0)
            
        if len(mountain_views) == 0:
            mountain_views.append(0)
            
        if len(summits) == 0:
            summits.append(0)
            
        if len(wildlife) == 0:
            wildlife.append(0)  
            
        if len(ridges) == 0:
            ridges.append(0)

        if len(campsites) == 0:
            campsites.append(0)
            
        if len(dogs_on_leash) == 0:
            dogs_on_leash.append(0)

        if len(no_dogs) == 0:
            no_dogs.append(0)

        if len(kid_friendly) == 0:
            kid_friendly.append(0)
            
        if len(parking_entry_fee) == 0:
            parking_entry_fee.append(None)
            
        report_link = url + '/@@related_tripreport_listing'
        report_html_rows = get_html_rows(report_link)
        report_date_list = []

        itr2 = -1
        for row in report_html_rows:
            itr2 += 1

            if '"count-data"' in row:
                report_count = int(report_html_rows[itr2 + 1].lstrip())
                report_counts.append(report_count)

            if '"elapsed-time"' in row:
                report_date = datetime.strptime(row[row.find('title="') + 7 : row.find('">')], '%b %d, %Y')
                report_date_list.append(report_date)

        if len(report_counts) != rownum:
            report_counts.append(None)

        if len(report_date_list) != 0:
            report_dates.append(report_date_list[0])
        elif len(report_dates) != rownum:
            report_dates.append(None)

        hike_links.append(url)
        
        if rownum % 100 == 0:
            print(str(rownum) + ' Hikes loaded...')
        rownum += 1
        
    print('Finished loading hikes!\n' + str(rownum - 1) + ' Hikes successfully loaded.') 
    print('titles: ', len(titles), ' entries')
    print('regions: ', len(regions), ' entries')
    print('specific region: ', len(specific_regions), ' entries')
    print('distances: ', len(distances), ' entries')
    print('dist_types: ', len(dist_types), ' entries')
    print('gains: ', len(gains), ' entries')
    print('highests: ', len(highests), ' entries')
    print('ratings: ', len(ratings), ' entries')
    print('rating_counts: ', len(rating_counts), ' entries')
    print('latitudes: ', len(latitudes), 'Entries')
    print('longitudes: ', len(longitudes), 'Entries')
    print('report_dates: ', len(report_dates), ' entries')
    print('report_counts: ', len(report_counts), ' entries')
    print('hike_links: ', len(hike_links), ' entries')
    print('coast: ', len(coast), " entries")
    
    return pd.DataFrame({'TITLE': titles, 'REGION': regions, 'SPECIFIC REGION': specific_regions, 'DISTANCE': distances,
                         'DIST_TYPE': dist_types, 'GAIN': gains, 'HIGHEST': highests,
                         'RATING OUT OF 5': ratings, 'RATING_COUNT': rating_counts, 
                         'LATITUDE': latitudes, 'LONGITUDE': longitudes, 
                         'MOST_RECENT_REPORT_DATE': report_dates, 'REPORT_COUNT': report_counts,
                         'COAST': coast, 'RIVERS': rivers, 'LAKES': lakes, 'WATERFALLS': waterfalls, 
                         'OLD GROWTH': old_growth, 'FALL FOLIAGE': fall_foliage,
                         'WILDFLOWERS/MEADOWS': wildflowers, 'MOUNTAIN VIEWS': mountain_views,
                         'SUMMITS': summits, 'WILDLIFE': wildlife, 'RIDGES': ridges, 'ESTABLISHED CAMPSITES': campsites,
                         'DOGS ALLOWED ON LEASH': dogs_on_leash, 'NO DOGS': no_dogs, 'KID FRIENDLY': kid_friendly,
                         'REQUIRED PASSES': parking_entry_fee,
                         'URL': hike_links})

In [6]:
# Taking a list of individual hike links as a parameter,
# this method will reutrn specific information for each hike in the format of a DataFrame.
def get_hike_info2(hike_urls):
    
    # Create the dataframe
    df = pd.DataFrame(columns=["TITLE", "REGION", "SPECIFIC REGION", "DISTANCE", "DIST_TYPE", "GAIN", "HIGHEST",
                          "RATING", "RATING_COUNT", "LATITUDE", "LONGITUDE", "MOST_RECENT_REPORT_DATE",
                          "REPORT_COUNT", "COAST", "RIVERS", "LAKES", "WATERFALLS", "OLD GROWTH",
                          "FALL FOLIAGE", "WILDFLOWERS/MEADOWS", "MOUNTAIN VIEWS", "SUMMITS",
                          "WILDLIFE", "RIDGES", "ESTABLISHED CAMPSITES", "DOGS ALLOWED ON LEASH",
                          "NO DOGS", "KID FRIENDLY", "REQUIRED PASSES", "URL"])


    rownum = 1
    for url in hike_urls:
        PARKING_PASS_FOUND = False
        
        titles = []
        regions = []
        specific_regions = []
        distances = []
        dist_types = []
        gains = []
        highests = []
        ratings = []
        rating_counts = []
        latitudes = []
        longitudes = []
        report_counts = []
        report_dates = []
        hike_links = []
        coast = []
        rivers = []
        lakes = []
        waterfalls = []
        old_growth = []
        fall_foliage = []
        wildflowers = []
        mountain_views = []
        summits = []
        wildlife = []
        ridges = []
        campsites = []
        dogs_on_leash = []
        no_dogs = []
        kid_friendly = []
        parking_entry_fee = []
        print(url)
        hike_html_rows = get_html_rows(url)

        itr1 = -1
        for row in hike_html_rows:
            itr1 += 1

            if '"documentFirstHeading"' in row:
                hike_title = hike_html_rows[itr1 + 1].lstrip()
                titles.append(hike_title)

            if 'Location' in row:
                hike_region_full = hike_html_rows[itr1 + 3].lstrip()
                if hike_region_full is not None:
                    hike_region = re.findall('([\s\S]*) --', hike_region_full)
                    if len(hike_region) == 0:
                        regions.append(hike_region_full)
                    else:
                        regions.append(hike_region[0])

                    specific_region = re.findall("-- ([\s\S]*)", hike_region_full)
                    if len(specific_region) == 0:
                        specific_regions.append(None)
                    else:
                        specific_regions.append(specific_region[0])
                
            # Looking for the other misc features
            if '"Coast"' in row:
                coast.append(1)
                    
            if '"Rivers"' in row:
                rivers.append(1)
                
            if '"Lakes"' in row:
                lakes.append(1)
                
            if '"Waterfalls"' in row:
                waterfalls.append(1)
                
            if '"Old growth"' in row:
                old_growth.append(1)
            
            if '"Fall foliage"' in row:
                fall_foliage.append(1)
                
            if '"Wildflowers/Meadows"' in row:
                wildflowers.append(1)

            if '"Mountain views"' in row:
                mountain_views.append(1)
                
            if '"Summits"' in row:
                summits.append(1)
                
            if '"Wildlife"' in row:
                wildlife.append(1)
                
            if '"Ridges/passes"' in row:
                ridges.append(1)
                
            if '"Established campsites"' in row:
                campsites.append(1)
                
            if '"Dogs allowed on leash"' in row:
                dogs_on_leash.append(1)
                
            if '"Dogs not allowed"' in row:
                no_dogs.append(1)
                
            if '"Good for kids"' in row:
                kid_friendly.append(1)

            if '"distance"' in row:
                hike_distance_string = hike_html_rows[itr1 + 2].lstrip()
                hike_distance = float(hike_distance_string[ : hike_distance_string.find(' mile')])
                if ',' in hike_distance_string:
                    hike_distance_type = hike_distance_string[hike_distance_string.find(', ') + 2 : ]
                elif 'of trails' in hike_distance_string:
                    hike_distance_type = hike_distance_string[hike_distance_string.find('of trails') + 3 : ]
                else:
                    hike_distance = 'ERROR'
                distances.append(hike_distance)
                dist_types.append(hike_distance_type)

            if 'Gain:' in row:
                hike_gain = float(hike_html_rows[itr1 + 2].lstrip())
                gains.append(hike_gain)

            if 'Highest Point:' in row:
                hike_highest = float(hike_html_rows[itr1 + 2].lstrip())
                highests.append(hike_highest)

            if '"current-rating"' in row:
                rating_string = hike_html_rows[itr1 + 1].lstrip()
                hike_rating = float(rating_string[ : rating_string.find(' out of')])
                ratings.append(hike_rating)

            if '"rating-count"' in row:
                rating_count_string = hike_html_rows[itr1 + 1].lstrip()
                rating_count = int(rating_count_string[rating_count_string.find('(') + 1 : rating_count_string.find(' vote')])
                rating_counts.append(rating_count)
                
            if '<script type="application/ld+json">' in row:
                json_string = hike_html_rows[itr1 + 1].lstrip()
                hike_json = json.loads(json_string)
                try:
                    latitude = hike_json['geo']['latitude']
                    longitude = hike_json['geo']['longitude']
                    latitudes.append(latitude)
                    longitudes.append(longitude)
                except:
                    pass
                
            if 'Parking Pass/Entry Fee' in row:
                if PARKING_PASS_FOUND == False:
                    PARKING_PASS_FOUND = True
                    required_pass = hike_html_rows[itr1 + 3].lstrip()
                    if required_pass == "None":
                        required_pass = "No pass"
                    parking_entry_fee.append(required_pass)

                
        if len(titles) != rownum:
            titles.append(None)

        if len(regions) != rownum:
            regions.append(None)
            
        if len(specific_regions) != rownum:
            specific_regions.append(None)

        if len(distances) != rownum:
            distances.append(None)

        if len(dist_types) != rownum:
            dist_types.append(None)

        if len(gains) != rownum:
            gains.append(None)

        if len(highests) != rownum:
            highests.append(None)

        if len(ratings) != rownum:
            ratings.append(None)

        if len(rating_counts) != rownum:
            rating_counts.append(None)
            
        if len(latitudes) != rownum:
            latitudes.append(None)

        if len(longitudes) != rownum:
            longitudes.append(None)
        
        if len(coast) == 0:
            coast.append(0)
            
        if len(rivers) == 0:
            rivers.append(0)
        
        if len(lakes) == 0:
            lakes.append(0)
        
        if len(waterfalls) == 0:
            waterfalls.append(0)
            
        if len(old_growth) == 0:
            old_growth.append(0)
            
        if len(fall_foliage) == 0:
            fall_foliage.append(0)
            
        if len(wildflowers) == 0:
            wildflowers.append(0)
            
        if len(mountain_views) == 0:
            mountain_views.append(0)
            
        if len(summits) == 0:
            summits.append(0)
            
        if len(wildlife) == 0:
            wildlife.append(0)  
            
        if len(ridges) == 0:
            ridges.append(0)

        if len(campsites) == 0:
            campsites.append(0)
            
        if len(dogs_on_leash) == 0:
            dogs_on_leash.append(0)

        if len(no_dogs) == 0:
            no_dogs.append(0)

        if len(kid_friendly) == 0:
            kid_friendly.append(0)
            
        if len(parking_entry_fee) == 0:
            parking_entry_fee.append(None)
            
        report_link = url + '/@@related_tripreport_listing'
        report_html_rows = get_html_rows(report_link)
        report_date_list = []

        itr2 = -1
        for row in report_html_rows:
            itr2 += 1

            if '"count-data"' in row:
                report_count = int(report_html_rows[itr2 + 1].lstrip())
                report_counts.append(report_count)

            if '"elapsed-time"' in row:
                report_date = datetime.strptime(row[row.find('title="') + 7 : row.find('">')], '%b %d, %Y')
                report_date_list.append(report_date)

        if len(report_counts) != rownum:
            report_counts.append(None)

        if len(report_date_list) != 0:
            report_dates.append(report_date_list[0])
        elif len(report_dates) != rownum:
            report_dates.append(None)

        hike_links.append(url)
        
        if rownum % 100 == 0:
            print(str(rownum) + ' Hikes loaded...')
        rownum += 1
        
        df = df.append({'TITLE': titles[0], 'REGION': regions[0], 'SPECIFIC REGION': specific_regions[0], 'DISTANCE': distances[0],
                         'DIST_TYPE': dist_types[0], 'GAIN': gains[0], 'HIGHEST': highests[0],
                         'RATING': ratings[0], 'RATING_COUNT': rating_counts[0], 
                         'LATITUDE': latitudes[0], 'LONGITUDE': longitudes[0], 
                         'MOST_RECENT_REPORT_DATE': report_dates[0], 'REPORT_COUNT': report_counts[0],
                         'COAST': coast[0], 'RIVERS': rivers[0], 'LAKES': lakes[0], 'WATERFALLS': waterfalls[0], 
                         'OLD GROWTH': old_growth[0], 'FALL FOLIAGE': fall_foliage[0],
                         'WILDFLOWERS/MEADOWS': wildflowers[0], 'MOUNTAIN VIEWS': mountain_views[0],
                         'SUMMITS': summits[0], 'WILDLIFE': wildlife[0], 'RIDGES': ridges[0], 'ESTABLISHED CAMPSITES': campsites[0],
                         'DOGS ALLOWED ON LEASH': dogs_on_leash[0], 'NO DOGS': no_dogs[0], 'KID FRIENDLY': kid_friendly[0],
                         'REQUIRED PASSES': parking_entry_fee[0],
                         'URL': hike_links[0]}, ignore_index=True)
        
    print('Finished loading hikes!\n' + str(rownum - 1) + ' Hikes successfully loaded.') 
    print('titles: ', len(titles), ' entries')
    print('regions: ', len(regions), ' entries')
    print('specific region: ', len(specific_regions), ' entries')
    print('distances: ', len(distances), ' entries')
    print('dist_types: ', len(dist_types), ' entries')
    print('gains: ', len(gains), ' entries')
    print('highests: ', len(highests), ' entries')
    print('ratings: ', len(ratings), ' entries')
    print('rating_counts: ', len(rating_counts), ' entries')
    print('latitudes: ', len(latitudes), 'Entries')
    print('longitudes: ', len(longitudes), 'Entries')
    print('report_dates: ', len(report_dates), ' entries')
    print('report_counts: ', len(report_counts), ' entries')
    print('hike_links: ', len(hike_links), ' entries')
    print('coast: ', len(coast), " entries")
    
    return df

In [7]:
# Get all hike page links.
all_hikes_list_pages = list(set().union(get_hikes_list_page_urls('https://www.wta.org/go-outside/hikes'), ['https://www.wta.org/go-outside/hikes']))

In [8]:
# Get all individual hike links.
all_individual_hikes = get_individual_hike_urls(all_hikes_list_pages)

In [9]:
# Get all hike data, initialize to DataFrame.
wta_hikes_df = get_hike_info2(all_individual_hikes)

https://www.wta.org/go-hiking/hikes/sawtooth-ridge-blue-mtns
https://www.wta.org/go-hiking/hikes/rattlesnake-trail
https://www.wta.org/go-hiking/hikes/oregon-butte
https://www.wta.org/go-hiking/hikes/mount-misery
https://www.wta.org/go-hiking/hikes/dexter-ridge
https://www.wta.org/go-hiking/hikes/middle-tie-trail-3240-to-middle-point-ridge-trail-3116
https://www.wta.org/go-hiking/hikes/grizzly-bear-ridge-3103
https://www.wta.org/go-hiking/hikes/crooked-creek
https://www.wta.org/go-hiking/hikes/tucannon-river-rattlesnake-loop
https://www.wta.org/go-hiking/hikes/packers-trail
https://www.wta.org/go-hiking/hikes/middle-point-ridge
https://www.wta.org/go-hiking/hikes/three-forks-3133
https://www.wta.org/go-hiking/hikes/packers-trail-mount-misery-loop
https://www.wta.org/go-hiking/hikes/turkey-creek
https://www.wta.org/go-hiking/hikes/tucannon-river
https://www.wta.org/go-hiking/hikes/third-creek-3100
https://www.wta.org/go-hiking/hikes/table-rock-squaw-peak
https://www.wta.org/go-hiking/hi

https://www.wta.org/go-hiking/hikes/rattlesnake-trail-1114
https://www.wta.org/go-hiking/hikes/burnt-mountain
https://www.wta.org/go-hiking/hikes/american-ridge
https://www.wta.org/go-hiking/hikes/bonaparte-lake-trail
https://www.wta.org/go-hiking/hikes/cabin-trail
https://www.wta.org/go-hiking/hikes/mount-bonaparte-circumnavigation
https://www.wta.org/go-hiking/hikes/grand-forest
https://www.wta.org/go-hiking/hikes/little-quilcene-river
https://www.wta.org/go-hiking/hikes/lake-angeles
https://www.wta.org/go-hiking/hikes/lily-lizard-lake-loop
https://www.wta.org/go-hiking/hikes/big-gulch-mukilteo
https://www.wta.org/go-hiking/hikes/spray-park
https://www.wta.org/go-hiking/hikes/langus-riverfront-park
https://www.wta.org/go-hiking/hikes/mount-adams-south-climb
https://www.wta.org/go-hiking/hikes/little-mashel-falls-bud-blancher-trail
https://www.wta.org/go-hiking/hikes/teneriffe-falls
https://www.wta.org/go-hiking/grand-valley-cameron-creek-loop
https://www.wta.org/go-hiking/hikes/mothe

https://www.wta.org/go-hiking/hikes/loch-katrine
https://www.wta.org/go-hiking/hikes/osprey-park
https://www.wta.org/go-hiking/hikes/growden-heritage-site
https://www.wta.org/go-hiking/hikes/trillium-community-forest-land
https://www.wta.org/go-hiking/hikes/northway-trail
https://www.wta.org/go-hiking/hikes/evergreen-state-college-organic-farm-trail
https://www.wta.org/go-hiking/hikes/old-timers-horse-trail
https://www.wta.org/go-hiking/hikes/colquhon-peak
https://www.wta.org/go-hiking/hikes/hidden-cove-trail
https://www.wta.org/go-hiking/hikes/south-skookum-lake-trail
https://www.wta.org/go-hiking/hikes/chelan-gorge-trail
https://www.wta.org/go-hiking/hikes/salsbury-point-park
https://www.wta.org/go-hiking/hikes/lily-point-marine-park-and-reserve
https://www.wta.org/go-hiking/hikes/whidbey-institute
https://www.wta.org/go-hiking/hikes/my-neighborhood
https://www.wta.org/go-hiking/hikes/chair-peak
https://www.wta.org/go-hiking/hikes/dandy-pass-road-snowshoe
https://www.wta.org/go-hikin

https://www.wta.org/go-hiking/hikes/mutton-mountain
https://www.wta.org/go-hiking/hikes/cripple-creek-falls
https://www.wta.org/go-hiking/hikes/lyman-lakes
https://www.wta.org/go-hiking/hikes/log-flume
https://www.wta.org/go-hiking/hikes/mad-lake
https://www.wta.org/go-hiking/hikes/loch-eileen
https://www.wta.org/go-hiking/hikes/lower-mad-river
https://www.wta.org/go-hiking/hikes/lakeview-ranch
https://www.wta.org/go-hiking/hikes/libby-lake
https://www.wta.org/go-hiking/hikes/little-giant
https://www.wta.org/go-hiking/hikes/lanham-lake
https://www.wta.org/go-hiking/hikes/leroy-creek-high-route
https://www.wta.org/go-hiking/hikes/lenore-lake-caves
https://www.wta.org/go-hiking/hikes/lake-elizabeth
https://www.wta.org/go-hiking/hikes/klahhane-ridge
https://www.wta.org/go-hiking/hikes/lake-constance
https://www.wta.org/go-hiking/hikes/kettle-crest-north
https://www.wta.org/go-hiking/hikes/45-mile-sheep-drive
https://www.wta.org/go-hiking/hikes/columbia-mountain-snowshoe
https://www.wta.or

https://www.wta.org/go-hiking/hikes/little-wenatchee-river-road-snowshoe
https://www.wta.org/go-hiking/hikes/loser-ridge-snowshoe
https://www.wta.org/go-hiking/hikes/lime-kiln-state-park
https://www.wta.org/go-hiking/hikes/little-spokane-river
https://www.wta.org/go-hiking/hikes/lake-valhalla-snowshoe
https://www.wta.org/go-hiking/hikes/licorice-fern
https://www.wta.org/go-hiking/hikes/lake-youngs
https://www.wta.org/go-hiking/hikes/lava-butte
https://www.wta.org/go-hiking/hikes/little-joe-lake
https://www.wta.org/go-hiking/hikes/little-divide
https://www.wta.org/go-hiking/hikes/lighting-creek-trail
https://www.wta.org/go-hiking/hikes/larch-mountain
https://www.wta.org/go-hiking/hikes/lake-twentytwo-snowshoe
https://www.wta.org/go-hiking/hikes/layser-cave
https://www.wta.org/go-hiking/hikes/lemei-lake
https://www.wta.org/go-hiking/hikes/little-bear-trail
https://www.wta.org/go-hiking/hikes/knox-creek
https://www.wta.org/go-hiking/hikes/lake-laura-backdoor
https://www.wta.org/go-hiking/

...

https://www.wta.org/go-hiking/hikes/sword-fern-trail
https://www.wta.org/go-hiking/hikes/river-to-rock-trail
https://www.wta.org/go-hiking/hikes/goldendale-observatory-state-park
https://www.wta.org/go-hiking/hikes/king-sisters-preserve
3300 Hikes loaded...
https://www.wta.org/go-hiking/hikes/briggs-lake
https://www.wta.org/go-hiking/hikes/jakles-lagoon
https://www.wta.org/go-hiking/hikes/turn-point-lighthouse
https://www.wta.org/go-hiking/hikes/rain-shadow-loop
https://www.wta.org/go-hiking/hikes/american-camp-and-south-beach
https://www.wta.org/go-hiking/hikes/westcott-bay-nature-reserve-and-sculpture-park
https://www.wta.org/go-hiking/hikes/umtanum-ridge-crest
https://www.wta.org/go-hiking/hikes/vendovi-island-trails
https://www.wta.org/go-hiking/hikes/sucia-island
https://www.wta.org/go-hiking/hikes/spencer-spit
https://www.wta.org/go-hiking/hikes/cussed-hollow
https://www.wta.org/go-hiking/hikes/dirtyface-lookout-site
https://www.wta.org/go-hiking/hikes/airport-levee-trail
https:/

https://www.wta.org/go-hiking/hikes/silver-shadow-trail
https://www.wta.org/go-hiking/hikes/indian-henrys-via-longmire
https://www.wta.org/go-hiking/hikes/fossil-trail
https://www.wta.org/go-hiking/hikes/lake-hills-greenbelt
https://www.wta.org/go-hiking/hikes/round-lake-wetland
https://www.wta.org/go-hiking/hikes/quartz-mountain-1
https://www.wta.org/go-hiking/hikes/reflection-lakes
https://www.wta.org/go-hiking/hikes/rainbow-loop
https://www.wta.org/go-hiking/hikes/preston-railroad-grade
https://www.wta.org/go-hiking/hikes/puget-power
https://www.wta.org/go-hiking/hikes/one-view-trail
https://www.wta.org/go-hiking/hikes/park-lake-side-canyon
https://www.wta.org/go-hiking/hikes/ocean-city-state-park
https://www.wta.org/go-hiking/hikes/nook-trail
https://www.wta.org/go-hiking/hikes/mt-wow
https://www.wta.org/go-hiking/hikes/mt-ararat
https://www.wta.org/go-hiking/hikes/mount-spokane-trail-100
https://www.wta.org/go-hiking/hikes/millersylvania-state-park
https://www.wta.org/go-hiking/hi

https://www.wta.org/go-hiking/hikes/mount-beljica
https://www.wta.org/go-hiking/hikes/roaring-creek
https://www.wta.org/go-hiking/hikes/manresa-grotto
https://www.wta.org/go-hiking/hikes/shepperds-dell-state-natural-area
https://www.wta.org/go-hiking/hikes/lester-ghost-town
https://www.wta.org/go-hiking/hikes/lake-christine
https://www.wta.org/go-hiking/hikes/east-peak-rattlesnake-trail
https://www.wta.org/go-hiking/hikes/clearwest-peak
https://www.wta.org/go-hiking/hikes/mckenzie-head
https://www.wta.org/go-hiking/hikes/cape-disappointment-state-park-cape-disappointment-lighthouse
3600 Hikes loaded...
https://www.wta.org/go-hiking/hikes/ilwaco-long-beach-discovery-trail
https://www.wta.org/go-hiking/hikes/fort-worden-state-park-point-wilson
https://www.wta.org/go-hiking/hikes/fort-ebey-state-park
https://www.wta.org/go-hiking/hikes/pioneer-park-la-conner
https://www.wta.org/go-hiking/hikes/hex-mountain-1
https://www.wta.org/go-hiking/hikes/south-whidbey-state-park
https://www.wta.org/

https://www.wta.org/go-hiking/hikes/lower-big-quilcene-river
https://www.wta.org/go-hiking/hikes/quick-creek-camp
https://www.wta.org/go-hiking/hikes/big-meadow-lake
https://www.wta.org/go-hiking/hikes/mcdowell-marsh-environmental-education-trail
https://www.wta.org/go-hiking/hikes/saltese-uplands
https://www.wta.org/go-hiking/hikes/carlisle-lake
https://www.wta.org/go-hiking/hikes/sweet-creek-falls-interpretive-trail
https://www.wta.org/go-hiking/hikes/blue-lake-3
https://www.wta.org/go-hiking/hikes/frenchmans-bar
https://www.wta.org/go-hiking/hikes/west-fork-foss-river-and-lakes
https://www.wta.org/go-hiking/hikes/west-crater
https://www.wta.org/go-hiking/hikes/west-butte
https://www.wta.org/go-hiking/hikes/wallace-falls
https://www.wta.org/go-hiking/hikes/vancouver-lake
https://www.wta.org/go-hiking/hikes/upper-lena-lake
https://www.wta.org/go-hiking/hikes/twisp-pass
https://www.wta.org/go-hiking/hikes/john-tursi-trail
https://www.wta.org/go-hiking/hikes/tunnel-creek-dosewallips-tra

https://www.wta.org/go-hiking/hikes/cowboy-mtn-5853
https://www.wta.org/go-hiking/hikes/county-line
https://www.wta.org/go-hiking/hikes/cowlitz-trail
https://www.wta.org/go-hiking/hikes/cispus-basin-cispus-pass
https://www.wta.org/go-hiking/hikes/clay-pit-road
https://www.wta.org/go-hiking/hikes/china-creek
https://www.wta.org/go-hiking/hikes/cooper-river
https://www.wta.org/go-hiking/hikes/coleman-weedpatch
https://www.wta.org/go-hiking/hikes/chopper-peak
https://www.wta.org/go-hiking/hikes/cleveland-mountain
Finished loading hikes!
3898 Hikes successfully loaded.
titles:  2  entries
regions:  2  entries
specific region:  2  entries
distances:  2  entries
dist_types:  2  entries
gains:  1  entries
highests:  2  entries
ratings:  2  entries
rating_counts:  2  entries
latitudes:  2 Entries
longitudes:  2 Entries
report_dates:  1  entries
report_counts:  2  entries
hike_links:  1  entries
coast:  1  entries


In [11]:
wta_hikes_df

,TITLE,REGION,SPECIFIC REGION,DISTANCE,DIST_TYPE,GAIN,HIGHEST,RATING,RATING_COUNT,LATITUDE,...,MOUNTAIN VIEWS,SUMMITS,WILDLIFE,RIDGES,ESTABLISHED CAMPSITES,DOGS ALLOWED ON LEASH,NO DOGS,KID FRIENDLY,REQUIRED PASSES,URL
0,Sawtooth Ridge,Eastern Washington,Palouse and Blue Mountains,8.0,roundtrip,500.0,5500.0,2.67,3,46.061784,...,1,0,0,1,0,0,0,1,No pass,https://www.wta.org/go-hiking/hikes/sawtooth-r...
1,Rattlesnake Trail,Eastern Washington,Palouse and Blue Mountains,10.0,roundtrip,2900.0,5700.0,3.43,7,46.204339,...,1,1,0,1,1,1,0,0,No pass,https://www.wta.org/go-hiking/hikes/rattlesnak...
2,Oregon Butte,Eastern Washington,Palouse and Blue Mountains,6.0,roundtrip,987.0,6387.0,4.14,7,46.118332,...,1,1,1,1,1,1,0,0,Northwest Forest Pass,https://www.wta.org/go-hiking/hikes/oregon-butte
3,Mount Misery,Eastern Washington,Palouse and Blue Mountains,16.0,roundtrip,1000.0,6401.0,2.75,4,46.118201,...,1,1,1,1,1,1,0,1,Northwest Forest Pass,https://www.wta.org/go-hiking/hikes/mount-misery
4,Dexter Ridge,Eastern Washington,Palouse and Blue Mountains,5.0,one-way,2547.0,5500.0,0.00,0,46.021522,...,0,0,0,0,0,0,0,0,Northwest Forest Pass,https://www.wta.org/go-hiking/hikes/dexter-ridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3893,China Creek,Issaquah Alps,None,0.8,one-way,None,None,3.40,5,47.533349,...,0,0,0,0,0,0,0,1,No pass,https://www.wta.org/go-hiking/hikes/china-creek
3894,Cooper River,Snoqualmie Region,Salmon La Sac/Teanaway,7.8,roundtrip,400.0,2800.0,3.36,11,47.409338,...,1,0,1,0,1,1,0,1,Northwest Forest Pass,https://www.wta.org/go-hiking/hikes/cooper-river
3895,Coleman Weedpatch,South Cascades,None,9.0,trails,1900.0,5712.0,2.60,5,46.422933,...,1,0,0,0,0,0,0,0,No pass,https://www.wta.org/go-hiking/hikes/coleman-we...
3896,Chopper Peak,Central Washington,Wenatchee,3.0,roundtrip,1440.0,2400.0,3.00,1,47.399304,...,0,1,0,0,0,0,0,0,None,https://www.wta.org/go-hiking/hikes/chopper-peak


In [10]:
# Write to csv file.
curr_date = datetime.now().date()
wta_hikes_df.to_csv('./Results/{0}_wta_hike_data.csv'.format(curr_date), index = False)
wta_hikes_df.to_json(r'./Results/{0}_wta_hike_data.json'.format(curr_date), 'records')

In [221]:
df = get_hike_info2(["https://www.wta.org/go-hiking/hikes/lake-margaret", "https://www.wta.org/go-hiking/hikes/red-bluff", 
                   "https://www.wta.org/go-hiking/hikes/burley-mountain", "https://www.wta.org/go-hiking/hikes/hegg-benson-trail",
                   "https://www.wta.org/go-hiking/hikes/north-tommy-ridge"])

https://www.wta.org/go-hiking/hikes/lake-margaret
https://www.wta.org/go-hiking/hikes/red-bluff
https://www.wta.org/go-hiking/hikes/burley-mountain
https://www.wta.org/go-hiking/hikes/hegg-benson-trail
https://www.wta.org/go-hiking/hikes/north-tommy-ridge
Finished loading hikes!
5 Hikes successfully loaded.
titles:  2  entries
regions:  2  entries
specific region:  2  entries
distances:  1  entries
dist_types:  1  entries
gains:  1  entries
highests:  1  entries
ratings:  2  entries
rating_counts:  2  entries
latitudes:  2 Entries
longitudes:  2 Entries
report_dates:  1  entries
report_counts:  2  entries
hike_links:  1  entries
coast:  1  entries


In [222]:
df

,TITLE,REGION,SPECIFIC REGION,DISTANCE,DIST_TYPE,GAIN,HIGHEST,RATING,RATING_COUNT,LATITUDE,...,MOUNTAIN VIEWS,SUMMITS,WILDLIFE,RIDGES,ESTABLISHED CAMPSITES,DOGS ALLOWED ON LEASH,NO DOGS,KID FRIENDLY,REQUIRED PASSES,URL
0,Lake Margaret,Central Cascades,None,26.2,one-way,10800.0,7300.0,0.00,0,None,...,1,0,0,1,0,0,0,0,Northwest Forest Pass,https://www.wta.org/go-hiking/hikes/lake-margaret
1,Red Bluff,Eastern Washington,Selkirk Range,9.0,roundtrip,220.0,3920.0,4.50,4,48.856356,...,0,0,1,0,1,1,0,1,None,https://www.wta.org/go-hiking/hikes/red-bluff
2,Burley Mountain,South Cascades,White Pass/Cowlitz River Valley,14.0,roundtrip,4000.0,5250.0,3.17,6,46.4388,...,1,1,0,0,0,0,0,1,None,https://www.wta.org/go-hiking/hikes/burley-mou...
3,Hegg-Benson Trail,North Cascades,North Cascades Highway - Hwy 20,1.2,roundtrip,None,None,0.00,0,48.492928,...,0,0,0,0,0,1,0,1,None,https://www.wta.org/go-hiking/hikes/hegg-benso...
4,North Tommy Trail,Central Cascades,None,None,None,None,None,2.33,3,47.939172,...,0,0,0,0,0,0,0,0,None,https://www.wta.org/go-hiking/hikes/north-tomm...
